In [1]:
import geopy.geocoders
from geopy.geocoders import Nominatim

geopy.geocoders.options.default_user_agent = 'my_app/2'
geopy.geocoders.options.default_timeout = None

geolocator = Nominatim(user_agent="skilled_nursing")

In [2]:
import requests
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim

In [3]:
import sqlalchemy

In [4]:
import json
import re

In [5]:
from sodapy import Socrata

MyAppToken = '5YNrrmFobthjbfVSPQWEZWkkk'
username = 'jennifer.faith16@gmail.com'
password = 'Nashua1998!'

# Connect to dataset
client = Socrata("data.cms.gov",
                 MyAppToken,
                 username,
                 password)

results = client.get("s2uc-8wxp", limit = 1000000)

In [6]:
# Convert to pandas DataFrame
df = pd.DataFrame.from_records(results)

In [7]:
small = df[['federal_provider_number', 'provider_name', 'provider_address',
    'provider_city', 'provider_state', 'provider_zip_code', 'county', 'geolocation']]

In [8]:
small['full_address']=small['provider_address'] + ', ' + small['provider_city'] + ', ' + small['provider_state'] + ', ' +  small['provider_zip_code']

<ipython-input-8-7a40433a1853>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  small['full_address']=small['provider_address'] + ', ' + small['provider_city'] + ', ' + small['provider_state'] + ', ' +  small['provider_zip_code']


In [9]:
small['geolocation'].isnull().sum()

39296

In [10]:
no_dup = small.drop_duplicates(subset = ['federal_provider_number'])

In [11]:
no_dup['geolocation'].isnull().sum()

941

In [49]:
# Fill location errors with 0s
# no_dup['geolocation']=no_dup['geolocation'].fillna(0)

In [22]:
no_dup = no_dup.reset_index()
no_dup.drop(columns='index', axis=1, inplace=True)
no_dup['geolocation'] = no_dup['geolocation'].fillna(0)

In [52]:
# def fill_geo(df):
#     for i in range(len(df)):
#         location = geolocator.geocode(df['full_address'][i])
#         if location==None:
#             df['geolocation']='error'
#         else:
#             df['geolocation'][i]=[location.latitude, location.longitude]
#     return df

In [53]:
# new_df = fill_geo(no_dup)

In [14]:
location = geolocator.geocode('501 YATES STREET, MOUNT VERNON, TX, 75457')
print(location.latitude, location.longitude)

33.184225 -95.229151


In [23]:
def geo_column_edit(dictionary):
    if dictionary==0:
        pass
    else:
        values = dictionary.values()
        values_list = list(values)
        return [values_list[1][1], values_list[1][0]]

In [24]:
no_dup['geolocation']=no_dup['geolocation'].apply(geo_column_edit)

In [15]:
# NOTE: 1,199 have bad addresses. 
no_dup[no_dup['geolocation'].isnull()].shape

(941, 9)

In [25]:
no_dup.head(5)

,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,county,geolocation,full_address
0,025039,MAPLE SPRINGS OF PALMER,12130 EAST MAPLE SPRINGS WAY,PALMER,AK,99645,Matanuska-Susitna,None,"12130 EAST MAPLE SPRINGS WAY, PALMER, AK, 99645"
1,045314,BARNES HEALTHCARE,1010 BARNES STREET,LONOKE,AR,72086,Lonoke,"[34.793023, -91.895455]","1010 BARNES STREET, LONOKE, AR, 72086"
2,045460,"COLONEL GLENN HEALTH AND REHAB, LLC",13700 DAVID O DODD ROAD,LITTLE ROCK,AR,72210,Pulaski,"[34.708313, -92.410304]","13700 DAVID O DODD ROAD, LITTLE ROCK, AR, 72210"
3,055304,DYCORA TRANSITIONAL HEALTH - QUAIL LAKE,1221 ROSEMARIE LANE,STOCKTON,CA,95207,San Joaquin,"[37.986818, -121.31983300000002]","1221 ROSEMARIE LANE, STOCKTON, CA, 95207"
4,055474,MAGNOLIA REHABILITATION & NURSING CENTER,8133 MAGNOLIA AVENUE,RIVERSIDE,CA,92504,Riverside,"[33.934686, -117.42148400000002]","8133 MAGNOLIA AVENUE, RIVERSIDE, CA, 92504"


In [26]:
no_dup.to_csv('nhs_descriptors.csv')

## ALL TRIAL CODE GOES BELOW HERE

In [39]:
location = geolocator.geocode('409 W. GLENOAKS BLVD., GLENDALE, CA, 91202')
print(location.latitude, location.longitude)

34.159914072873924 -118.2624365591205


In [58]:
# location = geolocator.geocode("175 5th Avenue NYC")
# print((location.latitude, location.longitude))

(40.741059199999995, -73.98964162240998)


In [19]:
nulls = no_dup[no_dup['geolocation'].isnull()]

In [22]:
nulls['geolocation']

0             PINE BLUFF, AR, 71603
2            LOS ANGELES, CA, 90061
5                   YUMA, CO, 80759
6                  EAGLE, CO, 81631
15        ATLANTIC BEACH, FL, 32233
                    ...            
625302           DIMMITT, TX, 79027
625334           CYPRESS, TX, 77429
625365            IRVING, TX, 75038
625527           HOUSTON, TX, 77048
625557             MIAMI, FL, 33185
Name: geolocation, Length: 1186, dtype: object

In [21]:
nulls['geolocation']= nulls['provider_city'] + ', ' +nulls['provider_state'] + ', ' + nulls['provider_zip_code']

<ipython-input-21-ed354414b01e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nulls['geolocation']= nulls['provider_city'] + ', ' +nulls['provider_state'] + ', ' + nulls['provider_zip_code']


In [23]:
def geo_null(address):
    location = geolocator.geocode(address)
    return [location.latitude, location.longitude]

In [ ]:
400:500

In [33]:
nulls['geolocation'].iloc[450:500] = nulls['geolocation'].iloc[450:500].apply(geo_null)

In [26]:
nulls['geolocation'].iloc[200:300]

100238     [30.9038252, -84.5754768]
104705     [39.7990175, -89.6439575]
105669      [42.128704, -87.8265089]
107603    [-32.7761075, -71.2007014]
107860      [40.797821, -85.8205411]
                     ...            
161901     [43.1810844, -95.8561126]
162594     [38.2319537, -97.7733809]
163435     [37.2397486, -96.9955919]
163961     [30.4943669, -92.4176324]
166540     [38.2856247, -85.8241312]
Name: geolocation, Length: 100, dtype: object

In [31]:
nulls['geolocation'].iloc[600:]

325239                  [41.2134073, -98.4580649]
325294                 [37.6126634, -114.5142613]
325307                 [42.6867264, -103.4133215]
325332                  [41.4486334, -97.7307364]
325333                  [38.5246441, -118.624578]
                           ...                   
625302                 [34.5488229, -102.3153053]
625334                  [29.9691116, -95.6971686]
625365                  [32.8295183, -96.9442177]
625527                  [29.7589382, -95.3676974]
625557    [25.72919456538001, -80.44483666022803]
Name: geolocation, Length: 586, dtype: object

In [ ]:
nulls_z = nulls[['federal_provider_number', 'geolocation']]

In [49]:
new = no_dup.merge(nulls_z, on = 'federal_provider_number')

In [50]:
new.drop(columns = ['geolocation_x'], axis=1, inplace=True)

In [51]:
new.head()

,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,county,full_address,geolocation_y
0,035242,CHINLE NURSING HOME,HIGHWAY 191 & HOSPITAL ROAD,CHINLE,AZ,86503,Apache,"HIGHWAY 191 & HOSPITAL ROAD, CHINLE, AZ, 86503","[36.16763243080752, -109.68806973451998]"
1,065309,WASHINGTON COUNTY NURSING HOME,676 W GREENHOUSE DR,AKRON,CO,80720,Washington,"676 W GREENHOUSE DR, AKRON, CO, 80720","[49.82296369811355, 24.514349484476256]"
2,115275,"A.G. RHODES HOME, INC, THE","350 BOULVARD, S.E.",ATLANTA,GA,30312,Fulton,"350 BOULVARD, S.E., ATLANTA, GA, 30312","[33.74310470677914, -84.37445028551396]"
3,145323,CARRIER MILLS NSG & REHAB CTR,6789 US RT 45,CARRIER MILLS,IL,62917,Saline,"6789 US RT 45, CARRIER MILLS, IL, 62917","[37.682745708232396, -88.62537025784138]"
4,145424,LANDMARK OF RICHTON PARK REHAB & NSG CTR,22660 SOUTH CICERO AVENUE,RICHTON PARK,IL,60471,Cook,"22660 SOUTH CICERO AVENUE, RICHTON PARK, IL, 6...","[41.48200539573889, -87.72284133679652]"
